In [298]:
import datetime as dt
import sqlite3
import os
import sys
from sqlite3 import Error
import pandas as pd
import re
import statistics

In [299]:
def limpiar_consola():
    os.system('cls' if os.name == 'nt' else 'clear')

In [300]:
class FechaError(Exception):
    def __init__(self, mensaje):
        self.mensaje = mensaje
        super().__init__(self.mensaje)
class CampoVacioERR(Exception):
    def __init__(self, mensaje):
        self.mensaje = mensaje
        super().__init__(self.mensaje)

def CampoVacio(cadena):
  if cadena.strip()=="":
    raise CampoVacioERR("Error: La cadena está vacía")
def validarFecha(fecha):
  if fecha.date() > dt.datetime.now().date():
    raise FechaError("Error: La FECHA SOBREPASA A LA FECHA ACTUAL DEL SISTEMA")

In [301]:
if not(os.path.isfile("Taller.db")):
  try:
    with sqlite3.connect("Taller.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
      c1 = conn.cursor()
      c1.execute("\
                CREATE TABLE SERVICIOS( \
                id_servicio INTEGER PRIMARY KEY, \
                nombre TEXT, \
                estado_servicio INTEGER,\
                costo REAL \
                )\
                ")
      c1.execute("\
                CREATE TABLE CLIENTES ( \
                id_clientes INTEGER PRIMARY KEY, \
                nombre TEXT, \
                estado_cliente INTEGER,\
                RFC TEXT, \
                CORREO TEXT \
                )\
                ")
      c1.execute("CREATE TABLE NOTAS(\
                folio INTEGER PRIMARY KEY,\
                id_clientes INTEGER,\
                estado INTEGER,\
                fecha timestap,\
                FOREIGN KEY(id_clientes) REFERENCES CLIENTES(id_clientes)\
                )"
              )
      c1.execute("\
                  CREATE TABLE DETALLES( \
                  id_detalles INTEGER PRIMARY KEY,\
                  folio INTEGER,\
                  id_servicio INTEGER,\
                  FOREIGN KEY(folio) REFERENCES NOTAS(folio)\
                  FOREIGN KEY(id_servicio) REFERENCES SERVICIOS(id_servicio)\
                  )\
                ")
  except Error as e:
    print(e)
  finally:
    if conn:
      conn.close()



In [302]:
def verificar_registros_existentes(conn):
    try:
            cur = conn.cursor()
            cur.execute("SELECT COUNT(*) FROM CLIENTES WHERE estado_cliente = 0")
            clientes_count = cur.fetchone()[0]
            cur.execute("SELECT COUNT(*) FROM SERVICIOS WHERE estado_servicio = 0")
            servicios_count = cur.fetchone()[0]

            if clientes_count > 0 and servicios_count > 0:
                return True
            else:
                return False

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
        return False

In [303]:
def mostrar_clientes(conn,actividad = 0):
    try:
            cur = conn.cursor()
            act = {"actividad":actividad}
            cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE estado_cliente = :actividad",act)
            clientes = cur.fetchall()
            if clientes:
                print("Lista de Clientes Registrados:")
                for cliente in clientes:
                    print(f"ID: {cliente[0]}, Nombre: {cliente[1]}")

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)

def mostrar_servicios(conn,actividad = 0):
    try:
        cur = conn.cursor()
        act = {"actividad":actividad}
        cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE estado_servicio = :actividad",act)
        servicios = cur.fetchall()
        if servicios:
            print("Lista de Servicios Registrados:")
            for servicio in servicios:
                print(f"ID: {servicio[0]}, Nombre: {servicio[1]}, Costo: {servicio[2]}")

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)

In [304]:
def registrar_nota(conn):
            cur = conn.cursor()
            while True:
                try:
                    while True:
                        try:
                            mostrar_clientes(conn)
                            id_cliente = int(input("Digite su identificador de cliente recuerde que debe estar previamente registrado: "))
                            valores = {"id_cliente": id_cliente}
                            cur.execute("SELECT id_clientes FROM CLIENTES WHERE id_clientes = :id_cliente AND estado_cliente = 0", valores)
                            bandera = cur.fetchone()
                            if bandera:
                                print("DATO AGREGADO EXITOSAMENTE")
                                break
                            else:
                                print("ESE CLIENTE NO SE ENCUENTRA REGISTRADO. INTENTE DE NUEVO")
                        except ValueError as ve:
                            print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
                        except sqlite3.Error as e:
                            print(e)

                    while True:
                        try:
                            fecha = input("Digite la fecha de esta nota dd/mm/yyyy: ")
                            fecha = dt.datetime.strptime(fecha, "%d/%m/%Y")
                            validarFecha(fecha)
                            break
                        except FechaError as FE:
                            print(FE)
                        except ValueError as ve:
                            print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
                        except Exception as E:
                            print(E)

                    valores = {"cliente": id_cliente, "fecha": fecha.date(), "estado": 0}
                    cur.execute("INSERT INTO NOTAS (id_clientes, estado, fecha) VALUES (:cliente, :estado, :fecha)", valores)
                    conn.commit()

                    try:
                        while True:
                            cur.execute("SELECT folio FROM NOTAS ORDER BY folio DESC LIMIT 1")
                            folio = cur.fetchone()

                            while True:
                                try:
                                    mostrar_servicios(conn)
                                    id_servicio = int(input("Digite la clave del servicio: "))
                                    valores = {"id_servicio": id_servicio}
                                    cur.execute("SELECT id_servicio FROM SERVICIOS WHERE id_servicio = :id_servicio AND estado_servicio = 0", valores)
                                    bandera = cur.fetchone()
                                    if bandera:
                                        print("DETALLE CREADO EXITOSAMENTE")
                                        break
                                    else:
                                        print("ESE SERVICIO NO EXISTE. INTENTE DE NUEVO")

                                except sqlite3.Error as e:
                                    print("USTED DIGITÓ UN VALOR INEXISTENTE")
                                except ValueError as ve:
                                    print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
                                except Exception as E:
                                    print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO", E)
                            while True:
                                try:
                                    salida = int(input("Digite 1 SI DESEA AGREGAR MÁS DETALLES Y 2 SI NO HAY MÁS POR AGREGAR:\n"))
                                    break
                                except ValueError as ve:
                                    print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")

                            valores = {"folio": folio[0], "servicio": id_servicio}
                            cur.execute("INSERT INTO DETALLES (folio, id_servicio) VALUES(:folio,:servicio)", valores)
                            conn.commit()
                            if salida == 2:
                                break
                                print("SALIDA SATISFACTORIA")

                    except sqlite3.Error as e:
                        print("USTED DIGITÓ UN VALOR INEXISTENTE")
                    except Exception as e:
                        print("OCURRIÓ UN ERROR. INTENTE DE NUEVO")
                    except CampoVacioERR as cv:
                        print(cv)

                    break

                except sqlite3.Error as e:
                    print(e)
                except ValueError as ve:
                    print(ve)


In [305]:
def cancelar_nota(conn):
  while True:
    try:
        cur = conn.cursor()

        while True:
          try:
            folio = int(input("Ingrese el folio de la nota a cancelar 0 para salir: "))
            break
          except Exception as e:
            print("ERROR EL VALOR DIGITADO NO CORESPONDE CON EL SOLICITADO VUELVA A INTENTAR")
            continue
        if folio == 0:
          print("SALIDA EXITOSA")
          return
        cur.execute("SELECT estado FROM NOTAS WHERE folio = ? AND estado = 0", (folio,))
        nota = cur.fetchone()
        if nota:
            cur.execute("SELECT n.folio, n.fecha, c.nombre AS cliente, s.nombre AS servicio, s.costo \
                        FROM NOTAS AS n \
                        JOIN CLIENTES AS c ON n.id_clientes = c.id_clientes \
                        JOIN DETALLES AS d ON n.folio = d.folio \
                        JOIN SERVICIOS AS s ON d.id_servicio = s.id_servicio \
                        WHERE n.folio = ?", (folio,))
            detalles = cur.fetchall()


            print("Detalles de la nota a cancelar:")
            for detalle in detalles:
                print(f"Folio: {detalle[0]}")
                print(f"Fecha: {detalle[1]}")
                print(f"Cliente: {detalle[2]}")
                print(f"Servicio: {detalle[3]}")
                print(f"Costo del Servicio: {detalle[4]}")
                print("-----------")

            while True:
              try:
                confirmacion = int(input("¿Desea cancelar esta nota? (1SI/2NO): "))
                break
              except Exception as e:
                print("ERROR EL VALOR DIGITADO NO CORREPONDE CON LO INDICADO INTENTE DE NUEVO")
                continue
            if confirmacion == 1:
                cur.execute("UPDATE NOTAS SET estado = 1 WHERE folio = ?", (folio,))
                conn.commit()
                print("La nota ha sido cancelada exitosamente.")
            else:
                print("La nota no ha sido cancelada.")
        else:
            print("La nota no existe o ya ha sido cancelada.")
    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)

    break




In [306]:

def recuperar_nota(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT folio, fecha FROM NOTAS WHERE estado = 1")
        notas_canceladas = cur.fetchall()
        if not notas_canceladas:
            print("No hay notas canceladas para recuperar.")
            return

        print("Listado de Notas Canceladas:")
        print("{:<10} {:<12}".format("Folio", "Fecha"))
        for nota in notas_canceladas:
            print("{:<10} {:<12}".format(nota[0], nota[1]))

        cur.execute("SELECT folio FROM NOTAS WHERE estado = 1")
        notas_canceladas = cur.fetchall()

        while True:
            try:
                folio_recuperar = int(input("Ingrese el folio de la nota a recuperar (o 0 para cancelar): "))
                if folio_recuperar == 0:
                    print("Operación cancelada.")
                    return
                if (folio_recuperar,) not in notas_canceladas:
                    print("El folio ingresado no corresponde a una nota cancelada o no existe.")
                else:
                    cur.execute("SELECT n.folio, n.fecha, c.nombre, c.RFC, c.CORREO, s.nombre, s.costo FROM NOTAS n "
                                "INNER JOIN CLIENTES c ON n.id_clientes = c.id_clientes "
                                "INNER JOIN DETALLES d ON n.folio = d.folio "
                                "INNER JOIN SERVICIOS s ON d.id_servicio = s.id_servicio "
                                "WHERE n.folio = :folio", {"folio": folio_recuperar})
                    nota_detalle = cur.fetchall()
                    if nota_detalle:
                        print("Detalle de la nota:")
                        print("{:<10} {:<12} {:<20} {:<15} {:<25} {:<20} {:<10}".format("Folio", "Fecha", "Cliente", "RFC", "Correo", "Servicio", "Costo"))
                        for detalle in nota_detalle:
                            print("{:<10} {:<12} {:<20} {:<15} {:<25} {:<20} {:<10}".format(*detalle))
                    else:
                        print("No se pudo recuperar el detalle de la nota.")
                        continue
                    while True:
                      try:
                        confirmacion = int(input("¿Desea recuperar esta nota? (1Sí/2No): "))
                        break
                      except ValueError:
                        print("El Valor no es un numero o no es valido")

                    if confirmacion == 1:
                        cur.execute("UPDATE NOTAS SET estado = 0 WHERE folio = :folio", {"folio": folio_recuperar})
                        conn.commit()
                        print("La nota ha sido recuperada exitosamente.")
                    else:
                        print("Operación cancelada.")
                    return
            except ValueError as ve:
                print("Por favor, ingrese un número de folio válido.")
            except sqlite3.Error as e:
                print("Error al acceder a la base de datos:", e)

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [307]:
def consulta_por_periodo(conn):
    try:
        cur = conn.cursor()
        while True:
          try:
            fecha_inicial_str = input("Ingrese la fecha inicial (dd/mm/yyyy) [01/01/2000 por defecto]: ").strip()
            if not fecha_inicial_str:
                fecha_inicial_str = "01/01/2000"
            fecha_inicial = dt.datetime.strptime(fecha_inicial_str, "%d/%m/%Y")
            validarFecha(fecha_inicial)
            break
          except Exception as e:
            print("Ocurrio un error a la hora de digitar las fechas recuerde que no puede sobre pasar a la del sistema intente de nuevo")

        while True:
          try:
            fecha_final_str = input("Ingrese la fecha final (dd/mm/yyyy) [Fecha actual por defecto]: ").strip()
            if not fecha_final_str:
                fecha_final = dt.datetime.now()
            else:
                fecha_final = dt.datetime.strptime(fecha_final_str, "%d/%m/%Y")
            if fecha_inicial > fecha_final:
              print("la fecha final es menor a la inicial vuelva intentar")
              continue
            break
          except FechaError as fe:
            print("Ocurrio un error la fecha sobre pasa a la del sistema favor de volver a intentar")
          except Exception as e:
            print("Ocurrio un error a la hora de digitar la fecha por favor intente de nuevo")


        cur.execute("SELECT NOTAS.folio, NOTAS.fecha, SUM(SERVICIOS.costo) AS MontoTotal \
                     FROM NOTAS \
                     JOIN CLIENTES ON NOTAS.id_clientes = CLIENTES.id_clientes \
                     JOIN DETALLES ON NOTAS.folio = DETALLES.folio \
                     JOIN SERVICIOS ON DETALLES.id_servicio = SERVICIOS.id_servicio \
                     WHERE NOTAS.fecha BETWEEN :fecha_inicial AND :fecha_final AND NOTAS.estado = 0 \
                     GROUP BY NOTAS.folio, NOTAS.fecha",
                    {"fecha_inicial": fecha_inicial, "fecha_final": fecha_final})
        notas_periodo = cur.fetchall()

        if not notas_periodo:
            print("No hay notas emitidas para el período especificado.")
            return

        df = pd.DataFrame(notas_periodo, columns=["Folio", "Fecha", "Monto Total"])


        cur.execute("SELECT AVG(detalles.total) FROM (SELECT notas.folio, SUM(servicios.costo) AS total "
                    "FROM notas JOIN detalles ON notas.folio = detalles.folio "
                    "JOIN servicios ON detalles.id_servicio = servicios.id_servicio "
                    "WHERE fecha BETWEEN :fecha_inicial AND :fecha_final "
                    "GROUP BY notas.folio) AS detalles", {"fecha_inicial": fecha_inicial, "fecha_final": fecha_final})
        monto_promedio = cur.fetchone()[0]
        print(f"Monto promedio de las notas del período: {monto_promedio:.2f}")
        while True:
            try:
              opcion = int(input("¿Desea exportar el reporte? (1(CSV/2(Excel/3(No): "))
              if opcion == 1:
                  nombre_archivo = f"ReportePorPeriodo_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.csv"
                  df.to_csv(nombre_archivo, index=False)
                  print(f"El reporte ha sido exportado a {nombre_archivo}")
                  break
              elif opcion == 2:
                  nombre_archivo = f"ReportePorPeriodo_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.xlsx"
                  df.to_excel(nombre_archivo, index=False)
                  print(f"El reporte ha sido exportado a {nombre_archivo}")
                  break
              elif opcion == 3:
                  break
              else:
                  print("Opción no válida. Por favor, ingrese '1', '2', o '3'.")
            except ValueError:
              print('usted digito un valor que no va acorde a lo que se le pidio')

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)



In [308]:
def consulta_por_folio(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT NOTAS.folio, NOTAS.fecha, CLIENTES.nombre \
                     FROM NOTAS \
                     JOIN CLIENTES ON NOTAS.id_clientes = CLIENTES.id_clientes \
                     WHERE NOTAS.estado = 0 \
                     ORDER BY NOTAS.folio")
        notas = cur.fetchall()

        if not notas:
            print("No hay notas disponibles para consultar.")
            return


        print("Listado de Notas Disponibles:")
        print("{:<10} {:<12} {:<20}".format("Folio", "Fecha", "Nombre del Cliente"))
        for nota in notas:
            print("{:<10} {:<12} {:<20}".format(nota[0], nota[1], nota[2]))
        while True:
          try:
            folio = int(input("Ingrese el folio de la nota a consultar (0 para volver al menú principal): "))
            break
          except Exception as e:
            print("el tipo de dato que usted digito es incorrecto intente de nuevo")
        if folio == 0:
            return

        cur.execute("SELECT NOTAS.folio, NOTAS.fecha, CLIENTES.nombre, CLIENTES.RFC, CLIENTES.CORREO, SERVICIOS.nombre, SERVICIOS.costo \
                     FROM NOTAS \
                     JOIN CLIENTES ON NOTAS.id_clientes = CLIENTES.id_clientes \
                     JOIN DETALLES ON NOTAS.folio = DETALLES.folio \
                     JOIN SERVICIOS ON DETALLES.id_servicio = SERVICIOS.id_servicio \
                     WHERE NOTAS.folio = :folio AND NOTAS.estado = 0", {"folio": folio})
        detalles = cur.fetchall()

        if not detalles:
            print("El folio no corresponde a una nota válida o la nota está cancelada.")
        else:
            print("\nDetalles de la Nota:")
            print("{:<10} {:<12} {:<20} {:<15} {:<25} {:<20} {:<10}".format("Folio", "Fecha", "Nombre del Cliente", "RFC", "Correo", "Servicio", "Costo"))
            monto_total = 0
            for detalle in detalles:
                print("{:<10} {:<12} {:<20} {:<15} {:<25} {:<20} {:<10}".format(detalle[0], detalle[1], detalle[2], detalle[3], detalle[4], detalle[5], detalle[6]))
                monto_total+=detalle[6]
            print(f"monto total:{monto_total}")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [309]:


def validar_rfc(rfc):
    rfc_pattern = re.compile(r'^([A-Z&Ññ]{3}|[A-Z][AEIOU][A-Z]{2})\d{2}((01|03|05|07|08|10|12)(0[1-9]|[12]\d|3[01])|02(0[1-9]|[12]\d)|(04|06|09|11)(0[1-9]|[12]\d|30))([A-Z0-9]{2}[0-9A])$', re.IGNORECASE)
    return bool(rfc_pattern.match(rfc))

def validar_correo(correo):
    correo_pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    return bool(correo_pattern.match(correo))

def registrar_cliente(conn):
    try:
        cur = conn.cursor()

        print("Ingrese los datos del cliente:")


        while True:
            nombre = input("Nombre completo del cliente: ").strip()
            if nombre:
                break
            else:
                print("El nombre no puede estar vacío o contener solo espacios en blanco.")

        while True:
            try:
              rfc = input("RFC del cliente:").strip()
              if validar_rfc(rfc):
                  break
              else:
                  print("RFC no válido. Verifique el formato.")
            except ValueError:
              print("El valor digitdo no es valido vuelva a intentar")

        while True:
            try:
              correo = input("Correo electrónico del cliente: ").strip()
              if validar_correo(correo):
                  break
              else:
                  print("Correo electrónico no válido. Verifique el formato.")
            except ValueError:
              print("El valor digitdo no es valido vuelva a intentar")


        valores = {"nombre": nombre, "rfc": rfc, "correo": correo}
        cur.execute("INSERT INTO CLIENTES (nombre, RFC, CORREO, estado_cliente) VALUES (:nombre, :rfc, :correo, 0)", valores)
        conn.commit()
        print("Cliente agregado exitosamente.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [310]:
def reporte_clientes_activos_por_clave(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE estado_cliente = 0")
        clientes = cur.fetchall()
        if clientes:
            print("Reporte de Clientes Activos ordenados por clave:")
            print("{:<10} {:<30} {:<15} {:<30}".format("Clave", "Nombre", "RFC", "Correo"))
            for cliente in clientes:
                print("{:<10} {:<30} {:<15} {:<30}".format(cliente[0], cliente[1], cliente[2], cliente[3]))

            while True:
                try:
                  exportar = int(input("¿Desea exportar el resultado a CSV, Excel o regresar al menú de reportes? (1CSV/2EXCEL/3Regresar): "))
                  if exportar in [1,2,3]:
                      break
                  else:
                      print("Opción no válida. Ingrese 1,2,3.")
                except ValueError:
                  print("El valor digitdo no es valido vuelva a intentar")

            if exportar == 1:
                fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")
                filename = f"ReporteClientesActivosPorClave_{fecha_actual}.csv"
                df = pd.DataFrame(clientes, columns=["Clave", "Nombre", "RFC", "Correo"])
                df.to_csv(filename, index=False)
                print(f"El reporte se ha exportado a {filename}")
            elif exportar == 2:
                fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")
                filename = f"ReporteClientesActivosPorClave_{fecha_actual}.xlsx"
                df = pd.DataFrame(clientes, columns=["Clave", "Nombre", "RFC", "Correo"])
                df.to_excel(filename, index=False)
                print(f"El reporte se ha exportado a {filename}")

        else:
            print("No hay clientes activos para mostrar en el reporte.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)




In [311]:
def reporte_clientes_activos_por_nombre(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE estado_cliente = 0 ORDER BY nombre")
        clientes = cur.fetchall()

        if clientes:
            print("Reporte de Clientes Activos ordenados por nombre:")
            print("{:<10} {:<30} {:<15} {:<30}".format("Clave", "Nombre", "RFC", "Correo"))
            for cliente in clientes:
                print("{:<10} {:<30} {:<15} {:<30}".format(cliente[0], cliente[1], cliente[2], cliente[3]))

            while True:
              try:
                exportar = int(input("¿Desea exportar el resultado a CSV, Excel o regresar al menú de reportes? (1CSV/2EXCEL/3Regresar): "))
                if exportar in [1,2,3]:
                    break
                else:
                    print("Opción no válida. Ingrese 1,2,3.")
              except Exception as e:
                print("El valor que digito no corresponde con el que se le pidio")

            if exportar == 1:
                fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")
                filename = f"ClientesActivosPorNombre_{fecha_actual}.csv"
                df = pd.DataFrame(clientes, columns=["Clave", "Nombre", "RFC", "Correo"])
                df.to_csv(filename, index=False)
                print(f"El reporte se ha exportado a {filename}")

            elif exportar == 2:
                fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")
                filename = f"ClientesActivosPorNombre_{fecha_actual}.xlsx"
                df = pd.DataFrame(clientes, columns=["Clave", "Nombre", "RFC", "Correo"])
                df.to_excel(filename, index=False)
                print(f"El reporte se ha exportado a {filename}")

        else:
            print("No hay clientes activos para mostrar en el reporte.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)





In [312]:
def agregar_servicio(conn):
    try:
        cur = conn.cursor()
        while True:
            nombre_servicio = input("Nombre del servcio: ").strip()
            if nombre_servicio:
                break
            else:
                print("El nombre no puede estar vacío o contener solo espacios en blanco.")


        while True:
          try:
              costo_servicio = input("Ingrese el costo del servicio: ")
              costo_servicio = float(costo_servicio)
              if costo_servicio <= 0:
                  print("El costo del servicio debe ser superior a 0.00.")
                  continue
              break
          except ValueError:
              print("El costo del servicio debe ser un número válido.")


        cur.execute("INSERT INTO SERVICIOS (nombre, costo, estado_servicio) VALUES (?, ?, 0)", (nombre_servicio, costo_servicio))
        conn.commit()
        print("El servicio se ha agregado exitosamente.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [313]:
def buscar_servicio_por_nombre(conn):
    try:
        cur = conn.cursor()
        nombre_buscar = input("Ingrese el nombre del servicio que desea buscar (ignorando mayúsculas/minúsculas): ")

        cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE UPPER(nombre) = UPPER(?) AND estado_servicio = 0", (nombre_buscar,))
        servicio = cur.fetchone()

        if servicio:
            print(f"Detalle del Servicio (Clave: {servicio[0]}):")
            print(f"Nombre: {servicio[1]}")
            print(f"Costo: {servicio[2]}")
        else:
            print(f"No se encontró un servicio con el nombre '{nombre_buscar}'.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [314]:
def buscar_servicio_por_clave(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS ORDER BY id_servicio AND estado_servicio = 0")
        servicios = cur.fetchall()

        if not servicios:
            print("No hay servicios registrados.")
            return

        print("Listado de Servicios por Clave:")
        for servicio in servicios:
            print(f"Clave: {servicio[0]}, Nombre: {servicio[1]}")
        while True:
          try:
            clave = int(input("Ingrese la clave del servicio que desea buscar: "))
            break
          except ValueError as vr:
            print("El valor que digito no es del mismo que del que se le pidio")

        for servicio in servicios:
            if servicio[0] == clave:
                print(f"Detalle del Servicio (Clave: {servicio[0]}):")
                print(f"Nombre: {servicio[1]}")
                print(f"Costo: {servicio[2]}")
                return

        print(f"No se encontró un servicio con la clave {clave}.")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [315]:
def generar_reporte_servicios_por_clave(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE estado_servicio = 0 ORDER BY id_servicio")
        servicios = cur.fetchall()
        if not servicios:
            print("No hay servicios registrados.")
            return
        print("Reporte de Servicios Activos ordenados por clave:")
        print("{:<10} {:<30} {:<15} ".format("Clave", "Nombre","Costo"))
        for servicio in servicios:
            print("{:<10} {:<30} {:<15}".format(servicio[0], servicio[1], servicio[2]))

        df = pd.DataFrame(servicios, columns=['Clave', 'Nombre', 'Costo'])
        fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")

        while True:
          try:
            opcion = int(input("¿Desea exportar el informe a CSV (1) o Excel (2)? (Presione 3 para omitir): "))
            if opcion in [1,2,3]:
                break
            else:
                print("Opción no válida. Ingrese 1,2,3.")
          except ValueError as ve:
            print("el valor no corresponde con el que usted digito")
        if opcion == 1:
            df.to_csv(f"ReporteServiciosPorClave_{fecha_actual}.csv", index=False)
            print("Informe exportado a CSV.")
        elif opcion == 2:
            df.to_excel(f"ReporteServiciosPorClave_{fecha_actual}.xlsx", index=False)
            print("Informe exportado a Excel.")
        else:
            print("Saliendo exitosamente")
            pass


    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)
    except Exception as e:
        print( e)


In [316]:
def generar_reporte_servicios_por_nombre(conn):
    try:
        cur = conn.cursor()
        cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE estado_servicio = 0 ORDER BY nombre")
        servicios = cur.fetchall()
        if not servicios:
            print("No hay servicios registrados.")
            return
        print("Reporte de Servicios Activos ordenados por clave:")
        print("{:<10} {:<30} {:<15}".format("Clave", "Nombre","Costo"))
        for servicio in servicios:
            print("{:<10} {:<30} {:<15}".format(servicio[0], servicio[1], servicio[2]))
        df = pd.DataFrame(servicios, columns=['Clave', 'Nombre', 'Costo'])
        fecha_actual = dt.datetime.now().strftime("%m_%d_%Y")


        while True:
          try:
            opcion = int(input("¿Desea exportar el informe a CSV (1) o Excel (2)? (Presione 3 para omitir): "))
            if opcion in [1,2,3]:
                break
            else:
                print("Opción no válida. Ingrese 1,2,3.")
            break
          except ValueError as ve:
            print("el valor digitado no corresponde con el que usted dio intente de nuevo")
        if opcion== 1:
            df.to_csv(f"ReporteServiciosPorNombre_{fecha_actual}.csv", index=False)
            print("Informe exportado a CSV.")
        elif opcion == 2:
            df.to_excel(f"ReporteServiciosPorNombre_{fecha_actual}.xlsx", index=False)
            print("Informe exportado a Excel.")
        else:
            print("salida exitosa")
            pass

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)


In [317]:


def buscar_clientes_por_nombre(conn):
    try:
        nombre = input("Ingrese el nombre del cliente a buscar: ")
        nombre = nombre.lower()
        cur = conn.cursor()
        cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE lower(nombre) LIKE :nombre AND estado_cliente = 0", {"nombre": f"%{nombre}%"})
        clientes = cur.fetchall()

        if not clientes:
            print("No se encontraron clientes con el nombre proporcionado.")
            return

        print("Clientes encontrados por nombre:")
        for cliente in clientes:
            print(f"ID: {cliente[0]}, Nombre: {cliente[1]}, RFC: {cliente[2]}, Correo: {cliente[3]}")

    except sqlite3.Error as e:
        print("Error al acceder a la base de datos:", e)




def buscar_clientes_por_clave(conn):
    try:
        while True:
            try:
                clave = int(input("Ingrese la clave del cliente a buscar (0 para volver al menú principal): "))
                if clave == 0:
                    return
                cur = conn.cursor()
                cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE id_clientes = :clave AND estado_cliente = 0", {"clave": clave})
                cliente = cur.fetchone()

                if not cliente:
                    print("No se encontró ningún cliente con la clave proporcionada.")
                else:
                    print("Cliente encontrado por clave:")
                    print(f"ID: {cliente[0]}, Nombre: {cliente[1]}, RFC: {cliente[2]}, Correo: {cliente[3]}")
                break
            except ValueError:
                print("Por favor, ingrese un número entero válido para la clave del cliente.")
            except sqlite3.Error as e:
                print("Error al acceder a la base de datos:", e)

    except Exception as e:
        print("Ocurrió un error:", e)


In [318]:
def suspender_cliente(conn):
  while True:
    try:
        cur = conn.cursor()
        mostrar_clientes(conn)
        while True:
          try:
            id = int(input("Ingrese el identificador del cliente a cancelar 0 para salir: "))
            break
          except Exception as e:
            print(e)
            continue
        if id == 0:
          print("SALIDA EXITOSA")
          return
        cur.execute("SELECT estado_cliente FROM CLIENTES WHERE id_clientes = ? AND estado_cliente = 0", (id,))
        cliente = cur.fetchone()
        if cliente:
            cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE id_clientes = ?", (id,))
            detalles = cur.fetchall()


            while True:
              try:
                confirmacion = int(input("¿Desea cancelar este cliente ? (1SI/2NO): "))
                break
              except Exception as e:
                print("POR FAVOR DIGITE UN VALOR NUMERICO")
                continue
            if confirmacion == 1:
                cur.execute("UPDATE CLIENTES SET estado_cliente = 1 WHERE id_clientes = ?", (id,))
                conn.commit()
                print("el cliente ha sido cancelado exitosamente.")
            else:
                print("el cliente no ha sido cancelado.")
        else:
            print("el cliente no existe o ya ha sido cancelado.")
    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    break

def recuperar_cliente(conn):
  while True:
    try:
        cur = conn.cursor()
        mostrar_clientes(conn,actividad=1)
        while True:
          try:
            id = int(input("Ingrese el identificador del cliente a recuperar 0 para salir: "))
            break
          except Exception as e:
            print(e)
            continue
        if id == 0:
          print("SALIDA EXITOSA")
          return
        cur.execute("SELECT estado_cliente FROM CLIENTES WHERE id_clientes = ? AND estado_cliente = 1", (id,))
        cliente = cur.fetchone()
        if cliente:
            cur.execute("SELECT id_clientes, nombre, RFC, CORREO FROM CLIENTES WHERE id_clientes = ?", (id,))
            detalles = cur.fetchall()


            while True:
              try:
                confirmacion = int(input("¿Desea recuperar este cliente? (1SI/2NO): "))
                break
              except Exception as e:
                print("porfavor digite un valor valido")
                continue
            if confirmacion == 1:
                cur.execute("UPDATE CLIENTES SET estado_cliente = 0 WHERE id_clientes = ?", (id,))
                conn.commit()
                print("el cliente ha sido recuperado exitosamente.")
            else:
                print("El cliente no ha sido recuperado")
        else:
            print("el cliente no existe o ya esta activo.")
    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    break

In [319]:
def suspender_servicio(conn):
  while True:
    try:
        cur = conn.cursor()
        mostrar_servicios(conn)
        while True:
          try:
            id = int(input("Ingrese el identificador del servicio a cancelar 0 para salir: "))
            break
          except Exception as e:
            print("porfavor digite un valor valido intente de nuevo")
            continue
        if id == 0:
          print("SALIDA EXITOSA")
          return
        cur.execute("SELECT estado_servicio FROM SERVICIOS WHERE id_servicio = ? AND estado_servicio = 0", (id,))
        cliente = cur.fetchone()
        if cliente:
            cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE id_servicio = ?", (id,))
            detalles = cur.fetchall()

            while True:
              try:
                confirmacion = int(input("¿Desea cancelar este servicio ? (1SI/2NO): "))
                break
              except Exception as e:
                print("POR FAVOR DIGITE UN VALOR NUMERICO")
                continue
            if confirmacion == 1:
                cur.execute("UPDATE SERVICIOS SET estado_servicio = 1 WHERE id_servicio = ?", (id,))
                conn.commit()
                print("el servicio ha sido cancelado exitosamente.")
            else:
                print("el servicio no ha sido cancelado.")
        else:
            print("el servicio no existe o ya ha sido cancelado.")
    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    break

def recuperar_servicio(conn):
  while True:
    try:
        cur = conn.cursor()
        mostrar_servicios(conn,actividad=1)
        while True:
          try:
            id = int(input("Ingrese el identificador del servicio a recuperar 0 para salir: "))
            break
          except Exception as e:
            print("Porafor digite un valor valido intente de nuevo")
            continue
        if id == 0:
          print("SALIDA EXITOSA")
          return
        cur.execute("SELECT estado_servicio FROM SERVICIOS WHERE id_servicio = ? AND estado_servicio = 1", (id,))
        cliente = cur.fetchone()
        if cliente:
            cur.execute("SELECT id_servicio, nombre, costo FROM SERVICIOS WHERE id_servicio = ?", (id,))
            detalles = cur.fetchall()


            while True:
              try:
                confirmacion = int(input("¿Desea recuperar este servicio ? (1SI/2NO): "))
                break
              except Exception as e:
                print("POR FAVOR DIGITE UN VALOR NUMERICO")
                continue
            if confirmacion == 1:
                cur.execute("UPDATE SERVICIOS SET estado_servicio = 0 WHERE id_servicio = ?", (id,))
                conn.commit()
                print("el servicio ha sido recuperado exitosamente.")
            else:
                print("el servicio no ha sido recuperado.")
        else:
            print("el servicio no existe o ya ha sido recuperado.")
    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)

    break


In [323]:
def servicios_mas_prestados(conn):
    try:
        cur = conn.cursor()

        while True:
          try:
            cantidad_servicios = int(input("Ingrese la cantidad de servicios más prestados a identificar: "))
            break
          except ValueError as v:
            print("EL VALOR NO CORRESPONDE AL QUE SE LE PIDIO")

        while True:
            try:
              fecha_inicial_str = input("Ingrese la fecha inicial del período a reportar (formato dd/mm/yyyy): ")
              fecha_inicial = dt.datetime.strptime(fecha_inicial_str, "%d/%m/%Y")
              validarFecha(fecha_inicial)
              fecha_final_str = input("Ingrese la fecha final del período a reportar (formato dd/mm/yyyy): ")
              fecha_final = dt.datetime.strptime(fecha_final_str, "%d/%m/%Y")
              validarFecha(fecha_final)
              if fecha_final < fecha_inicial:
                  print("La fecha final debe ser mayor o igual a la fecha inicial.")
                  continue
              break
            except ValueError as ve:
              print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
            except FechaError as FE:
              print(FE)
            except Exception as E:
              print(E)



        cur.execute("\
            SELECT SERVICIOS.nombre, COUNT(DETALLES.id_servicio) AS Cantidad\
            FROM DETALLES\
            JOIN SERVICIOS ON DETALLES.id_servicio = SERVICIOS.id_servicio\
            JOIN NOTAS ON DETALLES.folio = NOTAS.folio\
            WHERE NOTAS.fecha BETWEEN :fecha_inicial AND :fecha_final\
            GROUP BY SERVICIOS.id_servicio\
            ORDER BY Cantidad DESC\
            LIMIT :cantidad_servicios\
        ", {"fecha_inicial": fecha_inicial, "fecha_final": fecha_final, "cantidad_servicios": cantidad_servicios})
        resultados = cur.fetchall()
        if not resultados:
            print("No hay resultados para los parámetros proporcionados.")
            return


        df = pd.DataFrame(resultados, columns=["Servicio", "Cantidad"])
        print("\nServicios más prestados:")
        print(df)

        while True:
          try:
            opcion = int(input("¿Desea exportar el reporte a CSV (1) o a Excel (2)? (Ingrese 3 para omitir): "))
            if opcion == 1:
                df.to_csv(f"ReporteServiciosMasPrestados_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.csv", index=False)
                print("Reporte exportado a CSV.")
                break
            elif opcion == 2:
                df.to_excel(f"ReporteServiciosMasPrestados_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.xlsx", index=False)
                print("Reporte exportado a Excel.")
                break
            elif opcion == 3:
                break
            else:
                print("Opción no válida. Intente nuevamente.")
          except ValueError as ve:
            print("ERROR DIGITO UN VAOR INVALIDO INTENTE DE NUEVO")

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    except Exception as e:
        print("Ocurrió un error:", e)

def clientes_mas_notas(conn):
    try:
        cur = conn.cursor()

        while True:
          try:
            cantidad_clientes = int(input("Ingrese la cantidad de clientes a identificar: "))
            break
          except ValueError as v:
            print("EL VALOR NO CORRESPONDE AL QUE SE LE PIDIO")

        while True:
            try:
              fecha_inicial_str = input("Ingrese la fecha inicial del período a reportar (formato dd/mm/yyyy): ")
              fecha_inicial = dt.datetime.strptime(fecha_inicial_str, "%d/%m/%Y")
              validarFecha(fecha_inicial)
              fecha_final_str = input("Ingrese la fecha final del período a reportar (formato dd/mm/yyyy): ")
              fecha_final = dt.datetime.strptime(fecha_final_str, "%d/%m/%Y")
              validarFecha(fecha_final)
              if fecha_final < fecha_inicial:
                  print("La fecha final debe ser mayor o igual a la fecha inicial.")
                  continue
              break
            except ValueError as ve:
              print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
            except FechaError as FE:
              print(FE)
            except Exception as E:
              print(E)



        cur.execute("\
            SELECT CLIENTES.nombre, COUNT(NOTAS.id_clientes) AS Cantidad\
            FROM NOTAS\
            JOIN CLIENTES ON NOTAS.id_clientes = CLIENTES.id_clientes\
			      WHERE NOTAS.fecha BETWEEN :fecha_inicial AND :fecha_final\
            GROUP BY CLIENTES.id_clientes\
            ORDER BY Cantidad DESC\
            LIMIT :cantidad_clientes\
        ", {"fecha_inicial": fecha_inicial, "fecha_final": fecha_final, "cantidad_clientes": cantidad_clientes})
        resultados = cur.fetchall()
        if not resultados:
            print("No hay resultados para los parámetros proporcionados.")
            return


        df = pd.DataFrame(resultados, columns=["Cliente", "Cantidad"])
        print("\CLIENTES CON MAS NOTAS:")
        print(df)

        while True:
          try:
            opcion = int(input("¿Desea exportar el reporte a CSV (1) o a Excel (2)? (Ingrese 3 para omitir): "))
            if opcion == 1:
                df.to_csv(f"ReporteClientes_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.csv", index=False)
                print("Reporte exportado a CSV.")
                break
            elif opcion == 2:
                df.to_excel(f"ReporteClientes_{fecha_inicial.strftime('%m_%d_%Y')}_{fecha_final.strftime('%m_%d_%Y')}.xlsx", index=False)
                print("Reporte exportado a Excel.")
                break
            elif opcion == 3:
                break
            else:
                print("Opción no válida. Intente nuevamente.")
          except ValueError as ve:
            print("ERROR DIGITO UN VAOR INVALIDO INTENTE DE NUEVO")

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    except Exception as e:
        print("Ocurrió un error:", e)

def promedio_montos_notas(conn):
    try:
        cur = conn.cursor()

        while True:
          try:
            fecha_inicial_str = input("Ingrese la fecha inicial del período a reportar (formato dd/mm/yyyy): ")
            fecha_inicial = dt.datetime.strptime(fecha_inicial_str, "%d/%m/%Y")
            validarFecha(fecha_inicial)
            fecha_final_str = input("Ingrese la fecha final del período a reportar (formato dd/mm/yyyy): ")
            fecha_final = dt.datetime.strptime(fecha_final_str, "%d/%m/%Y")
            validarFecha(fecha_final)
            if fecha_final < fecha_inicial:
                print("La fecha final debe ser mayor o igual a la fecha inicial.")
                continue
            break
          except ValueError as ve:
            print("OCURRIÓ UN ERROR. EL VALOR QUE USTED PROPORCIONÓ NO CORRESPONDE AL QUE SE LE PIDIÓ. INTENTE DE NUEVO")
          except FechaError as FE:
            print(FE)
          except Exception as E:
            print(E)

        cur.execute("""
            SELECT AVG(total_costo)
            FROM (
            SELECT NOTAS.folio, SUM(SERVICIOS.costo) AS total_costo
            FROM DETALLES
            JOIN SERVICIOS ON DETALLES.id_servicio = SERVICIOS.id_servicio
            JOIN NOTAS ON DETALLES.folio = NOTAS.folio
            WHERE NOTAS.fecha BETWEEN :fecha_inicial AND :fecha_final
            GROUP BY NOTAS.folio
    ) AS subconsulta
        """, {"fecha_inicial": fecha_inicial, "fecha_final": fecha_final})


        resultado = cur.fetchone()

        if not resultado:
            print("No hay resultados para los parámetros proporcionados.")
            return

        print(f"\nMonto promedio de las notas:${resultado[0]:,.2f}")

    except sqlite3.Error as e:
        print("Ocurrió un error al acceder a la base de datos:", e)
    except Exception as e:
        print("Ocurrió un error:", e)


In [ ]:


try:
  with sqlite3.connect("Taller.db", detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
    cur = conn.cursor()
    while True:
      try:
        MENU_PRINCIPAL = int(input("------- ESTA EN EL MENU PRINCIPAL A QUE AREA DESEA IR -------\n1)NOTAS\n2)CLIENTES\n3)SERVICIOS\n4)SALIR\n5)ESTADISTICOS\n"))


        if MENU_PRINCIPAL == 5:
          while True:
            try:
              MENU_STATS = int(input("------- ESTA EN EL MENU DE ESTADISTICAS A QUE AREA DESEA IR ---------\n1)MAS VENDIDOS\n2)PROMEDIO\n3)SALIR\n4)Clientes mas notas\n"))
              if MENU_STATS == 1:
                servicios_mas_prestados(conn)
              if MENU_STATS == 2:
                promedio_montos_notas(conn)
              if MENU_STATS == 4:
                clientes_mas_notas(conn)
              if MENU_STATS == 3:
                break
            except ValueError as VE:
              print("OCURRIO UN ERROR VUELVA INTENER")


        if MENU_PRINCIPAL == 1:
          while True:
            try:
              MENU_NOTAS = int(input("------- ESTA EN EL MENU NOTAS A QUE AREA DESEA IR -------\n1)REGISTRAR UNA NOTA\n2)CANCELAR UNA NOTA\n3)RECUPERAR UNA NOTA\n4)CONSULTAS Y REPORTES\n5)SALIR\n"))
              if MENU_NOTAS == 1:
                acceso = verificar_registros_existentes(conn)
                if acceso:
                  registrar_nota(conn)
                else:
                  print("NO EXISTEN DATOS PARA PROCEDER A CREAR UNA NOTA")

              if MENU_NOTAS == 2:
                cancelar_nota(conn)
                continue

              if MENU_NOTAS == 3:
                recuperar_nota(conn)
                continue

              if MENU_NOTAS == 4:
                while True:
                  try:
                    MENU_CONSULTAS = int(input("------- ESTA EN EL MENU NOTAS A QUE AREA DESEA IR -------\n1)CONSULTA POR FOLIO\n2)CONSULTA POR PERIODO\n3)SALIR"))
                    if MENU_CONSULTAS == 1:
                      consulta_por_folio(conn)
                      continue
                    if MENU_CONSULTAS == 2:
                      consulta_por_periodo(conn)
                      continue
                    if MENU_CONSULTAS == 3:
                      break
                  except Exception as e:
                    print("ERROR SE LE PIDIO UN VALOR DISTINTO AL QUE DIO INTENTE DE NUEVO",e)
              if MENU_NOTAS == 5:
                print("SALIENDO DEL MENU DE NOTAS")
                break
            except ValueError as VE:
              print("El valor que usted digito no corresponde con lo que se le pidio")












        if MENU_PRINCIPAL == 3:
          while True:
            try:
              MENU_SERVICIOS = int(input("------- ESTA EN EL MENU SERVICIOS A QUE AREA DESEA IR ---re----\n1)AGREGAR SERVICIO\n2)MENU CONSULTAS\n3)SALIR\n4)Suspender servicios\n5)Recuperar servicio\n"))
              if MENU_SERVICIOS == 4:
                suspender_servicio(conn)
              if MENU_SERVICIOS == 5:
                recuperar_servicio(conn)
                pass
              if MENU_SERVICIOS == 1:
                agregar_servicio(conn)
              if MENU_SERVICIOS == 2:
                while True:
                  try:
                     MENU_CONSULTAS = int(input("------- ESTA EN EL MENU CONSULTAS A QUE AREA DESEA IR -------\n1)MENU DE CONSULTAS OREDENADAS\n2)BUSQUEDA DE SERVICIOS POR CLAVE\n3)BUSQUEDA DE SEREVICIO POR NOMBRE\n4)SALIR\n"))
                     if MENU_CONSULTAS == 1:
                      while True:
                        try:
                          MENU_ORDENAMIENTO = int(input("------- ESTA EN EL MENU ORDENAMINETO A QUE AREA DESEA IR -------\n1)ORDENADO POR CLAVE\n2)ORDENADOS POR NOMBRE DE SERVICIO\n3)SALIR\n"))
                          if MENU_ORDENAMIENTO == 1:
                            generar_reporte_servicios_por_clave(conn)
                          if MENU_ORDENAMIENTO == 2:
                            generar_reporte_servicios_por_nombre(conn)
                          if MENU_ORDENAMIENTO == 3:
                            break
                        except Exception as e:
                          print("ERROR El valor digitado no es acorde a lo que se le pidio Intente de nuevo ")
                     if MENU_CONSULTAS == 2:
                      buscar_servicio_por_clave(conn)
                      pass
                     if MENU_CONSULTAS == 3:
                      buscar_servicio_por_nombre(conn)
                      pass
                     if MENU_CONSULTAS == 4:
                      break
                  except Exception as e:
                    print("ocurrio un error intente de nuevo")
              if MENU_SERVICIOS == 3:
                break
            except Exception as e:
              print("ocurrio un error intente de nuevo")


        if MENU_PRINCIPAL == 2:
          while True:
            try:
              MENU_CLIENTES = int(input("------- ESTA EN EL MENU DE CLIENTES A QUE AREA DESEA IR ---------\n1)AGREGAR CLIENTE\n2)MENU CONSULTAS\n3)SALIR\n4)SUPENDER CLIENTE\n5)RECUPERAR CLIENTE\n"))
              if MENU_CLIENTES == 1:
                registrar_cliente(conn)
              if MENU_CLIENTES == 4:
                suspender_cliente(conn)
                pass
              if MENU_CLIENTES == 5:
                recuperar_cliente(conn)
                pass

              if MENU_CLIENTES == 2:
                while True:
                  try:
                     MENU_CONSULTAS = int(input("------- ESTA EN EL MENU CONSULTAS A QUE AREA DESEA IR -------\n1)MENU DE CONSULTAS OREDENADAS\n2)BUSQUEDA DE CLIENTES POR CLAVE\n3)BUSQUEDA DE CLIENTES POR NOMBRE\n4)SALIR\n"))
                     if MENU_CONSULTAS == 1:
                      while True:
                        try:
                          MENU_ORDENAMIENTO = int(input("------- ESTA EN EL MENU ORDENAMINETO A QUE AREA DESEA IR -------\n1)ORDENAR POR CLAVE\n2)ORDENAR POR NOMBRE\n3)SALIR\n"))
                          if MENU_ORDENAMIENTO == 1:
                            reporte_clientes_activos_por_clave(conn)
                          if MENU_ORDENAMIENTO == 2:
                            reporte_clientes_activos_por_nombre(conn)
                          if MENU_ORDENAMIENTO == 3:
                            break
                        except Exception as e:
                          print("el valor digitado no corresponde con el que se le pidio digite de nuevo",e)

                     if MENU_CONSULTAS == 2:
                      buscar_clientes_por_clave(conn)
                      pass
                     if MENU_CONSULTAS == 3:
                      buscar_clientes_por_nombre(conn)
                      pass
                     if MENU_CONSULTAS == 4:
                      break

                  except Exception as e:
                    print("el valor digitado no corresponde con el que se le pidio digite de nuevo")
              if MENU_CLIENTES == 3:
                break
            except Exception as e:
              print("EL VALOR DIGITADO NO CORRESPONDE CON EL QUE DIO INTENDE DE NUEVO")


        if MENU_PRINCIPAL == 4:
          print("SALIENDO DEL MENU PRINCIPAL")
          break
      except ValueError as VE:
        print("el valor digitado no corresponde con el que se le pidio digite de nuevo")

except sqlite3.Error as e:
    print(e)
finally:
    if conn:
      conn.close()